<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Préparation-des-données" data-toc-modified-id="Préparation-des-données-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Préparation des données</a></span></li><li><span><a href="#Modèle" data-toc-modified-id="Modèle-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Modèle</a></span></li><li><span><a href="#TP" data-toc-modified-id="TP-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>TP</a></span></li></ul></div>

Quelques liens utiles pour la suite 

[Tutoriel rapide Skorch](https://skorch.readthedocs.io/en/latest/user/tutorials.html) : faire au moins celui du "Basic Usage"

[Skorch FAQ : How do I shuffle my train batches](https://skorch.readthedocs.io/en/latest/user/FAQ.html#how-do-i-shuffle-my-train-batches)

[Skorch FAQ : I want to use sample_weight, how can I do this](https://skorch.readthedocs.io/en/latest/user/FAQ.html#i-want-to-use-sample-weight-how-can-i-do-this)

Ce notebook est basé sur le tutoriel pytorch ["Classifying Names with a Character-Level RNN"](https://pytorch.org/tutorials/intermediate/char_rnn_classification_tutorial.html)

In [ ]:
USE_CUDA = False # Ne mettez à true que si vous pensez avoir un GPU relativement puissant

# Préparation des données

Les cellules suivantes sont extraites du tutoriel

In [ ]:
from __future__ import unicode_literals, print_function, division
from io import open
import glob
import os

def findFiles(path): return glob.glob(path)

print(findFiles('data/names/*.txt'))

In [ ]:
import unicodedata
import string

all_letters = string.ascii_letters + " .,;'"
n_letters = len(all_letters)

In [ ]:
# Turn a Unicode string to plain ASCII, thanks to http://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )

In [ ]:
print(unicodeToAscii('Ślusàrski'))

In [ ]:
# Build the category_lines dictionary, a list of names per language
category_lines = {}
all_categories = []

# Read a file and split into lines
def readLines(filename):
    lines = open(filename, encoding='utf-8').read().strip().split('\n')
    return [unicodeToAscii(line) for line in lines]

In [ ]:
for filename in findFiles('data/names/*.txt'):
    category = os.path.splitext(os.path.basename(filename))[0]
    all_categories.append(category)
    lines = readLines(filename)
    category_lines[category] = lines

n_categories = len(all_categories)

In [ ]:
# Compte le nombre de noms par langue, et triez en fonction
sorted([(lang,len(category_lines[lang])) for lang in all_categories],key=lambda xy: xy[1],reverse=True)

## OneHotEncoding et padding des données

(À partir d'ici, ce n'est plus le tutoriel pytorch)

In [ ]:
# Calcul de la longueur maximale d'un nom
MAX_LETTERS= max(reduce(lambda x,y : x+y,[[len(l) for l in category_lines[d]] for d in category_lines]))

In [ ]:
from sklearn import preprocessing
# "le" sert à encoder les langages ("Arabic" => 0, "Chinese" => 1 etc.)
le = preprocessing.LabelEncoder()
le.fit(all_categories)

# letterEncoder sert à one-hot-encoder les lettres 
# https://fr.wikipedia.org/wiki/Encodage_one-hot
letterEncoder = preprocessing.LabelBinarizer()
letterEncoder.fit(list(all_letters))

In [ ]:
# Fonction qui "remplit" les matrices des mots onehotencodés 
# avec des nan jusqu'à la longueur MAX_LETTERS
def lineToPaddedTensor(line):
    # transformation en float32 pour pytorch/gpu
    lencoded = letterEncoder.transform(np.array(list(line)).reshape(-1,1)).astype(np.float32)
    padded = np.pad(lencoded,((0,MAX_LETTERS-len(line)),(0,0)),'constant',constant_values=(np.nan,))
    return padded

In [ ]:
letterEncoder.inverse_transform(lineToPaddedTensor('truc'))

In [ ]:
# fonction qui prend en argument un dictionnaire { <language1> : [<liste de noms>], etc.} et 
# renvoie une entrée X avec les noms one-hot-encodés et la sortie y (label) coresspondant
def dictToSamples(d):
    tuplesList = reduce(lambda x,y: x + y,[[(y,x) for y in d[x]] for x in d.keys() ])
    X = np.array(list(map(lambda xy: lineToPaddedTensor(xy[0]),tuplesList)))
    # le type longlong est requis pour les labels dans pytorch
    y = le.transform(list(map(lambda xy: xy[1],tuplesList))).astype(np.longlong) 
    return X,y

In [ ]:
X,y = dictToSamples(category_lines)

In [ ]:
numsample = 10000
letterEncoder.inverse_transform(X[numsample]),le.inverse_transform([y[numsample]])

In [ ]:
X.shape # X est un tenseur (nombre de noms, nombre de lettres, taille de l'alphabet)

# Modèle

C'est exactement le même modèle que celui du tutoriel, transformé en modèle standard "machine learning" (entrée tensorielle)

In [ ]:
# Fonction pour compter le nombre de paramètres entraînables dans le modèle
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [ ]:
import torch.nn as nn
F = nn.functional

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()

        self.hidden_size = hidden_size
        self.output_size = output_size
        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)

    def forward_internal(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        return output, hidden

    def forward(self, input): # ici l'entrée est "batchée"
        batch_size = input.size()[0] # taille du batch
        max_letters = input.size()[1] # taille du mot
        # on initialize le vecteur des sorties cachées
        # si on est sur gpu il faut l'initialiser sur le gpu
        if USE_CUDA:
            hidden = torch.cuda.FloatTensor(batch_size, self.hidden_size).fill_(0)
        else:
            hidden = torch.zeros(batch_size, self.hidden_size)
        # on boucle sur l'ensemble des lettres
        for letter in range(max_letters):
            # optimisation, si tous les mots du batch n'ont plus de lettres "réelles" on s'arrète
            # car le reste est du padding
            if torch.isnan(input[:,letter]).any(): 
                break
            # et finalement on effectue le forward RNN proprement dit,
            # notez qu'on ajoute un axe à la liste des lettres batchées
            # pour le "combined" (première ligne du forward_internal)
            output, hidden = self.forward_internal(input[:,letter].view(batch_size,-1), hidden)
        return output
        
n_hidden = 128
rnn = RNN(n_letters, n_hidden, n_categories)
count_parameters(rnn)

In [ ]:
from skorch import NeuralNetClassifier,callbacks
import torch

# décommenter pour réinitialiser rnn
# rnn = RNN(n_letters, n_hidden, n_categories)

# Utilitaire pour afficher une barre de progression
# pendant l'entraînement
callbacksT = [callbacks.ProgressBar(detect_notebook=False)]

# Le classifieur skorch
net = NeuralNetClassifier(
        rnn, # c'est notre modèle
        device=('cuda' if USE_CUDA else 'cpu'), #cf. remarque au début
        max_epochs=1, # on se limite à un parcours complet de tous les exemples dans l'ordre
        batch_size=1, # un échantillon par batch
        train_split=None, # on ne fait pas de validation, donc pas de split
        lr=0.005, # conseillé pour l'instant
        criterion=torch.nn.CrossEntropyLoss, # softmax + nnloss, cas multiclasse simple label
        callbacks = callbacksT
    )

In [ ]:
# Entraînement
net.fit(X,y)

In [ ]:
# Fonction utilitaire qui permet de "lisser" une liste de points 
# (en l'occurence les coûts d'entraînement)
def smooth_curve(points, factor=0.9):
    smoothed_points = []
    for point in points:
        if smoothed_points:
            previous = smoothed_points[-1]
            smoothed_points.append(previous * factor + point * (1 - factor))
        else:
            smoothed_points.append(point)
    return smoothed_points

In [ ]:
# Fonction pour l'affichage des courbes d'entraînement (train_loss)

import matplotlib.pyplot as plt

# prend en argument, un classifieur skorch, le nom du score à afficher, par défaut le coût d'entraînement
def plot_train_scores(net,score_name='train_loss',smoothing=0.999):
    train_scores = reduce(lambda x,y: x+y,net.history[:,'batches',:,score_name])
    plt.figure()
    plt.plot(smooth_curve(train_scores,smoothing))
    plt.show();

In [ ]:
# fonction pour afficher la matrice de confusion
from sklearn.metrics import confusion_matrix
import matplotlib.ticker as ticker

# prend en argument les entrées, les sorties(classes) et le classifieur
def plot_confusion_matrix(X,y,net):
    cm = confusion_matrix(y,net.predict(X))
    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    
    fig = plt.figure()
    ax = fig.add_subplot(111)
    cax = ax.matshow(cm)
    fig.colorbar(cax)

    # Set up axes
    ax.set_xticklabels([''] + all_categories, rotation=90)
    ax.set_yticklabels([''] + all_categories)

    # Force label at every tick
    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show();

# TP

### A) Premier problème
1. Afficher la courbe d'entraînement et la matrice de confusion, que constate-t-on ? D'où vient le problème ? (un indice se trouve dans l'un des liens au début)
2. Donner la solution (qui nécessite juste un paramètre supplémentaire dans le classifieur) et justifier.
 
### B) Second problème
1. Refaire la courbe d'entraînement et la matrice avec la solution trouvée. Qualifier la courbe d'entraînement par rapport à la précédente.
2. Afficher la matrice de confusion : qu'observe-t-on? D'où vient le problème? Comparer avec le code de l'entraînement du tutoriel pytorch original, que fait-il de différent? (indice : nombre de mots par langue) S'aider d'un moteur de recherche pour trouver la bonne dénomination pour qualifier cette démarche. (indice : commence par "stratified" en anglais). Justifier.

### C) Troisième problème
1. Consulter l'aide de la fonction sklearn : ```compute_class_weight```. L'utiliser avec le mode automatique sur notre jeu de données, qu'obtient-on?
2. Consulter le lien sur la FAQ skorch concernant l'application des ```sample_weight``` (code fourni dans les cellules ci-dessous). Expliquer ce que fait la nouvelle fonction ```get_loss``` ci-dessous, tout en complétant le code manquant dans les fonctions ```get_loss``` et dans ```unreduced_loss```, qui effectue l'opération inverse de ```get_loss```, mais sur un seul example. (Attention, contrairement au code donné dans la FAQ skorch, on utilise ici les poids sur les classes et non pas sur les échantillons).
3. Expliquer pourquoi cette démarche est grosso-modo équivalente à celle du tutoriel original et pourquoi elle résoud le problème précédent.
4. Expliquer pourquoi il faut ajouter le paramètre ```criterion__reduction='none'``` au classifieur (consultez la documentation de ```CrossEntropyLoss``` dans pytorch)
5. Afficher sur la même graphique les courbes du coût réduit et non-réduit, commenter. 
6. Contrôler et commenter la matrice de confusion. Comparer avec celle du tutoriel original, quel paramètre simple d'entraînement pourrait-on changer pour obtenir le même niveau de qualité?

     
### D) Quatrième problème
1. Consulter la documentation de ```GridSearchCV``` sur ```scikit-learn```. Executer GridSearchCV avec une recherche sur les paramètres suivants, en prenant bien soin de préciser le scoring :
    - le learning rate lr
    - la taille du batch 
    - le nombre de neurones cachés dans le RNN
2. Donner le meilleur résultat trouvé par le GridSearchCV, donner le taux de réussite sur le jeu de données et comparer avec le tutoriel original.

In [ ]:
class WeightedNeuralNetClassifier(NeuralNetClassifier):
    def __init__(self,*args,**kwargs):
        self.class_weights = torch.tensor(kwargs.pop('class_weigths').astype(np.float32))
        super().__init__(*args,**kwargs)
        
    def get_loss(self, y_pred, y_true, X, *args, **kwargs):
        # override get_loss to use the sample_weight from X
        loss_unreduced = super().get_loss(y_pred, y_true, X, *args, **kwargs)
        loss_reduced = # calculer ici le nouveau coût et le moyenner avec .mean() (on est sur un batch)
        return loss_reduced

# Cette fonction permet de recalculer le coût original
def unreduced_loss(net,X=None, y=None):
    loss_reduced = net.history[-1,'batches',-1,'train_loss']
    loss_unreduced = # calculer le coût original, ne pas moyenner
    return loss_unreduced

In [ ]:
rnn = RNN(n_letters, n_hidden, n_categories)

callbacksT = [callbacks.ProgressBar(detect_notebook=False),
              # On enregistre la fonction de coût original dans l'historique
              callbacks.BatchScoring(unreduced_loss,on_train=True,name='real_train_loss')]

# Attention il manque ici un paramètre, (celui du premier problème)
net = WeightedNeuralNetClassifier(
        rnn,
        class_weigths = weights_balanced,
        device=('cuda' if USE_CUDA else 'cpu'),
        max_epochs=1,
        batch_size=1,
        train_split=None,
        lr=0.005,
        criterion=torch.nn.CrossEntropyLoss,
        callbacks = callbacksT,
        criterion__reduction='none'
    )

In [ ]:
train_scores = smooth_curve(reduce(lambda x,y: x+y,net.history[:,'batches',:,'train_loss']),0.999)
real_train_scores = smooth_curve(reduce(lambda x,y: x+y,net.history[:,'batches',:,'real_train_loss']),0.999)